## Extracción y transformación de datos con Spark 2.4
### Origen de datos
Los datos representan los accidentes de tráfico por día y distrito
que se han producido en la ciudad de Madrid entre 2010 y 2018

El fichero origen es un fichero csv por año tiene los datos detallados
por cada uno de los intervinientes en el accidente: Conductor, pasajero,
testito, peatón, ...

La primera parte del fichero está referida al accidente y se repite
por cada uno de los intervinientes.

### La parte del registro referida al accidente tiene los campos:
 - 'FECHA'
 - 'RANGO HORARIO'
 - 'DIA SEMANA'
 - 'DISTRITO'
 - 'LUGAR ACCIDENTE'
 - 'Nº: string'
 - 'Nº PARTE'     **Id.accidente, es el mismo para varios regs.**
 - 'CPFA Granizo'
 - 'CPFA Hielo'
 - 'CPFA Lluvia'
 - 'CPFA Niebla'
 - 'CPFA Seco'
 - 'CPFA Nieve'
 - 'CPSV Mojada'
 - 'CPSV Aceite'
 - 'CPSV Barro'
 - 'CPSV Grava Suelta'
 - 'CPSV Hielo'
 - 'CPSV Seca Y Limpia'
 - '* Nº VICTIMAS' **Total de víctimas por Nº de parte**
 - 'TIPO ACCIDENTE'
 - 'Tipo Vehiculo'
 
### La parte del registro correspondiente a cada persona:
 - 'TIPO PERSONA'
 - 'SEXO'
 - 'LESIVIDAD'
 - 'Tramo Edad'


In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [3]:
accidenteData.printSchema()

root
 |-- FECHA: string (nullable = true)
 |-- RANGO HORARIO: string (nullable = true)
 |-- DIA SEMANA: string (nullable = true)
 |-- DISTRITO: string (nullable = true)
 |-- LUGAR ACCIDENTE: string (nullable = true)
 |-- Nº: string (nullable = true)
 |-- Nº PARTE: string (nullable = true)
 |-- CPFA Granizo: string (nullable = true)
 |-- CPFA Hielo: string (nullable = true)
 |-- CPFA Lluvia: string (nullable = true)
 |-- CPFA Niebla: string (nullable = true)
 |-- CPFA Seco: string (nullable = true)
 |-- CPFA Nieve: string (nullable = true)
 |-- CPSV Mojada: string (nullable = true)
 |-- CPSV Aceite: string (nullable = true)
 |-- CPSV Barro: string (nullable = true)
 |-- CPSV Grava Suelta: string (nullable = true)
 |-- CPSV Hielo: string (nullable = true)
 |-- CPSV Seca Y Limpia: string (nullable = true)
 |-- * Nº VICTIMAS: string (nullable = true)
 |-- TIPO ACCIDENTE: string (nullable = true)
 |-- Tipo Vehiculo: string (nullable = true)
 |-- TIPO PERSONA: string (nullable = true)
 |-- S

In [18]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
  .filter ((func.substring(accidenteData.LESIVIDAD,1,1)=='H') | 
            (func.substring(accidenteData.LESIVIDAD,1,1)=='M') | 
            (accidenteData['TIPO PERSONA'] == 'CONDUCTOR'))\
  .select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy').alias('fecha'), \
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        accidenteData["TIPO PERSONA"].alias('tipoPersona'), \
        accidenteData["SEXO"].alias('sexo'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [19]:
#Las columnas del Dataframe accRed quedan así
accRed.show()

+----------+--------------------+-----------+--------------------+------+---------+
|     fecha|            distrito|idAccidente|         tipoPersona|  sexo|lesividad|
+----------+--------------------+-----------+--------------------+------+---------+
|2018-01-01|USERA            ...|     2018/1|PEATON           ...|HOMBRE|       HG|
|2018-01-01|USERA            ...|     2018/1|           CONDUCTOR|HOMBRE|       IL|
|2018-01-01|HORTALEZA        ...|     2018/3|           CONDUCTOR|HOMBRE|       IL|
|2018-01-01|HORTALEZA        ...|     2018/3|VIAJERO          ...|HOMBRE|       HG|
|2018-01-01|SAN BLAS         ...|     2018/9|           CONDUCTOR|HOMBRE|       IL|
|2018-01-01|SAN BLAS         ...|     2018/9|           CONDUCTOR|HOMBRE|       IL|
|2018-01-01|SAN BLAS         ...|     2018/9|VIAJERO          ...| MUJER|       HL|
|2018-01-01|CIUDAD LINEAL    ...|    2018/12|           CONDUCTOR|HOMBRE|       HG|
|2018-01-01|CIUDAD LINEAL    ...|    2018/12|           CONDUCTOR|HOMBRE|   

In [35]:
# Eliminamos las filas de las personas ilesas
# Agrupamos por fecha, distrito, accidente y lesividad para contar 
# los registros y tener el total de afectados.
accGrouped=accRed\
                .groupBy('fecha','distrito','idAccidente')\
                .agg(func.count(func.when(accRed.lesividad=='MT',1)).alias('muertos'),
                     func.count(func.when(accRed.lesividad=='HG',1)).alias('graves'),
                     func.count(func.when(accRed.lesividad=='HL',1)).alias('leves'),
                     func.count(func.when((accRed.sexo == 'MUJER') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condMujer'),
                     func.count(func.when((accRed.sexo == 'HOMBRE') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condHombre'),
                    )\
                .na.fill(0)\
                .sort ('fecha','distrito','idAccidente')

In [36]:
#Las columnas del Dataframe accRed quedan así
accGrouped.toPandas()

,fecha,distrito,idAccidente,muertos,graves,leves,condMujer,condHombre
0,2018-01-01,ARGANZUELA,2018/25,0,0,1,0,2
1,2018-01-01,CARABANCHEL,2018/23,0,1,0,0,2
2,2018-01-01,CENTRO,2018/76,0,0,2,0,1
3,2018-01-01,CHAMARTIN,2018/43,0,0,1,0,2
4,2018-01-01,CIUDAD LINEAL,2018/12,0,1,2,0,2
5,2018-01-01,FUENCARRAL-EL PARDO,2018/16,0,0,2,0,1
6,2018-01-01,HORTALEZA,2018/3,0,1,0,0,1
7,2018-01-01,MORATALAZ,2018/8,0,0,1,0,2
8,2018-01-01,RETIRO,2018/34,0,0,2,0,2
9,2018-01-01,SALAMANCA,2018/20,0,1,1,1,1


In [37]:
accRes = accGrouped\
   .groupBy (accGrouped.fecha, accGrouped.distrito)\
   .agg (
            func.count(func.when(accGrouped.condMujer==0,1)).alias('condHombre'),
            func.count(func.when(accGrouped.condHombre==0,1)).alias('condMujer'),
            func.count(func.when((accGrouped.condMujer>0)&
                                 (accGrouped.condHombre>0),1)).alias('condMixto'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.muertos)).alias('muertosCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.graves)).alias('gravesCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.leves)).alias('levesCondHombre'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.muertos)).alias('muertosCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.graves)).alias('gravesCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.leves)).alias('levesCondMujer'),
            func.sum (accGrouped.muertos).alias('totalMuertos'),
            func.sum (accGrouped.graves).alias('totalGraves'),
            func.sum (accGrouped.leves).alias('totalLeves')
        )\
    .na.fill(0)\
    .orderBy (accGrouped.fecha, accGrouped.distrito)

In [38]:
accRes.toPandas()

,fecha,distrito,condHombre,condMujer,condMixto,muertosCondHombre,gravesCondHombre,levesCondHombre,muertosCondMujer,gravesCondMujer,levesCondMujer,totalMuertos,totalGraves,totalLeves
0,2018-01-01,ARGANZUELA,1,0,0,0,0,1,0,0,0,0,0,1
1,2018-01-01,CARABANCHEL,1,0,0,0,1,0,0,0,0,0,1,0
2,2018-01-01,CENTRO,1,0,0,0,0,2,0,0,0,0,0,2
3,2018-01-01,CHAMARTIN,1,0,0,0,0,1,0,0,0,0,0,1
4,2018-01-01,CIUDAD LINEAL,1,0,0,0,1,2,0,0,0,0,1,2
5,2018-01-01,FUENCARRAL-EL PARDO,1,0,0,0,0,2,0,0,0,0,0,2
6,2018-01-01,HORTALEZA,1,0,0,0,1,0,0,0,0,0,1,0
7,2018-01-01,MORATALAZ,1,0,0,0,0,1,0,0,0,0,0,1
8,2018-01-01,RETIRO,1,0,0,0,0,2,0,0,0,0,0,2
9,2018-01-01,SALAMANCA,0,0,1,0,0,0,0,0,0,0,1,1


#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [12]:
accGrouped.toPandas().to_csv('/home/ubuntu/Downloads/victimas_por_fecha_distrito_accidente_lesion.csv',sep=';')